<a href="https://colab.research.google.com/github/amitadhainje/LearningBERT/blob/master/TFX_and_BERT_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install tfx==0.22.0
!pip install tensorflow_text

#Installing the required modules
!pip install bert-for-tf2
!pip install sentencepiece
print ("The modules like bert-for-tf2 and sentencepiece is installed")

The modules like bert-for-tf2 and sentencepiece is installed


In [2]:
import glob
import os
import pprint
import re
import tempfile
from shutil import rmtree

import tfx
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_data_validation as tfdv
import tensorflow_hub as hub
import tensorflow_model_analysis as tfma
import tensorflow_transform as tft
import tensorflow_transform.beam as tft_beam
from tensorflow_transform.beam.tft_beam_io import transform_fn_io
from tensorflow_transform.saved import saved_transform_io
from tensorflow_transform.tf_metadata import (dataset_metadata, dataset_schema,
                                              metadata_io, schema_utils)
from tfx.components import (Evaluator, ExampleValidator, ImportExampleGen,
                            ModelValidator, Pusher, ResolverNode, SchemaGen,
                            StatisticsGen, Trainer, Transform)
from tfx.components.base import executor_spec
from tfx.components.trainer.executor import GenericExecutor
from tfx.dsl.experimental import latest_blessed_model_resolver
from tfx.proto import evaluator_pb2, example_gen_pb2, pusher_pb2, trainer_pb2
from tfx.types import Channel
from tfx.types.standard_artifacts import Model, ModelBlessing
from tfx.utils.dsl_utils import external_input

import tensorflow_datasets as tfds
import tensorflow_model_analysis as tfma
import tensorflow_text as text

from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext

%load_ext tfx.orchestration.experimental.interactive.notebook_extensions.skip

In [3]:
print('TensorFlow version: {}'.format(tf.__version__))
print('TFX version: {}'.format(tfx.__version__))

TensorFlow version: 2.2.0
TFX version: 0.22.0


In [4]:
context = InteractiveContext()

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
!ls -ltr /content/drive/'My Drive'/'Colab Notebooks'/NLP/tf_records/

total 8560
-rw-------+ 1 root root 6111169 Jun 18 04:28 imdb_train_data.tfrecords
-rw-------+ 1 root root 2653375 Jun 18 04:28 imdb_test_data.tfrecords


In [8]:
output = example_gen_pb2.Output(
             split_config=example_gen_pb2.SplitConfig(splits=[
                 example_gen_pb2.SplitConfig.Split(name='train', hash_buckets=45),
                 example_gen_pb2.SplitConfig.Split(name='eval', hash_buckets=5)
             ]))
# Load the data from our prepared TFDS folder
examples = external_input("/content/drive/My Drive/Colab Notebooks/NLP/tf_records/")
example_gen = ImportExampleGen(input=examples, output_config=output)

context.run(example_gen)

ExecutionResult(
    component_id: ImportExampleGen
    execution_id: 1
    outputs:
        examples: Channel(
            type_name: Examples
            artifacts: [Artifact(type_name: Examples, uri: /tmp/tfx-interactive-2020-06-18T06_48_24.897236-a9e6u4mw/ImportExampleGen/examples/1, id: 2)]
        ))

In [9]:
%%skip_for_export

for artifact in example_gen.outputs['examples'].get():
    print(artifact.uri)

/tmp/tfx-interactive-2020-06-18T06_48_24.897236-a9e6u4mw/ImportExampleGen/examples/1
This cell will be skipped during export to pipeline.


In [10]:
%%skip_for_export

statistics_gen = StatisticsGen(
    examples=example_gen.outputs['examples'])
context.run(statistics_gen)

context.show(statistics_gen.outputs['statistics'])

Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


This cell will be skipped during export to pipeline.


In [11]:
%%skip_for_export

schema_gen = SchemaGen(
    statistics=statistics_gen.outputs['statistics'],
    infer_feature_shape=True)
context.run(schema_gen)

context.show(schema_gen.outputs['schema'])

,Type,Presence,Valency,Domain
Feature name,,,,
'review',BYTES,required,,-
'sentiment',INT,required,,-


This cell will be skipped during export to pipeline.


In [12]:
%%skip_for_export
%%writefile bert.py

import tensorflow_hub as hub

BERT_TFHUB_URL = "https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/2"

def load_bert_layer(model_url=BERT_TFHUB_URL):
    # Load the pre-trained BERT model as layer in Keras
    bert_layer = hub.KerasLayer(
        handle=model_url,
        trainable=True)
    return bert_layer

Overwriting bert.py
This cell will be skipped during export to pipeline.


In [13]:
%%skip_for_export
%%writefile transform.py

import tensorflow as tf
import tensorflow_text as text

import tensorflow_hub as hub

BERT_TFHUB_URL = "https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/2"

def load_bert_layer(model_url=BERT_TFHUB_URL):
    # Load the pre-trained BERT model as layer in Keras
    bert_layer = hub.KerasLayer(
        handle=model_url,
        trainable=True)
    return bert_layer

MAX_SEQ_LEN = 64  # max number is 512
do_lower_case = load_bert_layer().resolved_object.do_lower_case.numpy()

def preprocessing_fn(inputs):
    """Preprocess input column of text into transformed columns of.
        * input token ids
        * input mask
        * input type ids
    """

    CLS_ID = tf.constant(101, dtype=tf.int64)
    SEP_ID = tf.constant(102, dtype=tf.int64)
    PAD_ID = tf.constant(0, dtype=tf.int64)

    vocab_file_path = load_bert_layer().resolved_object.vocab_file.asset_path
    
    bert_tokenizer = text.BertTokenizer(vocab_lookup_table=vocab_file_path, 
                                        token_out_type=tf.int64, 
                                        lower_case=do_lower_case) 
    
    def tokenize_text(text, sequence_length=MAX_SEQ_LEN):
        """
        Perform the BERT preprocessing from text -> input token ids
        """

        # convert text into token ids
        tokens = bert_tokenizer.tokenize(text)
        
        # flatten the output ragged tensors 
        tokens = tokens.merge_dims(1, 2)[:, :sequence_length]
        
        # Add start and end token ids to the id sequence
        start_tokens = tf.fill([tf.shape(text)[0], 1], CLS_ID)
        end_tokens = tf.fill([tf.shape(text)[0], 1], SEP_ID)
        tokens = tokens[:, :sequence_length - 2]
        tokens = tf.concat([start_tokens, tokens, end_tokens], axis=1)

        # truncate sequences greater than MAX_SEQ_LEN
        tokens = tokens[:, :sequence_length]

        # pad shorter sequences with the pad token id
        tokens = tokens.to_tensor(default_value=PAD_ID)
        pad = sequence_length - tf.shape(tokens)[1]
        tokens = tf.pad(tokens, [[0, 0], [0, pad]], constant_values=PAD_ID)

        # and finally reshape the word token ids to fit the output 
        # data structure of TFT  
        return tf.reshape(tokens, [-1, sequence_length])

    def preprocess_bert_input(text):
        """
        Convert input text into the input_word_ids, input_mask, input_type_ids
        """
        input_word_ids = tokenize_text(text)
        input_mask = tf.cast(input_word_ids > 0, tf.int64)
        input_mask = tf.reshape(input_mask, [-1, MAX_SEQ_LEN])
        
        zeros_dims = tf.stack(tf.shape(input_mask))
        input_type_ids = tf.fill(zeros_dims, 0)
        input_type_ids = tf.cast(input_type_ids, tf.int64)

        return (
            input_word_ids, 
            input_mask,
            input_type_ids
        )

    input_word_ids, input_mask, input_type_ids = \
        preprocess_bert_input(tf.squeeze(inputs['review'], axis=1))

    return {
        'input_word_ids': input_word_ids,
        'input_mask': input_mask,
        'input_type_ids': input_type_ids,
        'sentiment': inputs['sentiment']
    }

Overwriting transform.py
This cell will be skipped during export to pipeline.


In [14]:
transform = Transform(
    examples=example_gen.outputs['examples'],
    schema=schema_gen.outputs['schema'],
    module_file=os.path.abspath("transform.py"))
context.run(transform)

Instructions for updating:
Schema is a deprecated, use schema_utils.schema_from_feature_spec to create a `Schema`
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use ref() instead.


  name: "review"
  type: BYTES
  presence {
    min_fraction: 1.0
    min_count: 1
  }
  shape {
    dim {
      size: 1
    }
  }
}
feature {
  name: "sentiment"
  type: INT
  bool_domain {
  }
  presence {
    min_fraction: 1.0
    min_count: 1
  }
  shape {
    dim {
      size: 1
    }
  }
}
}) belongs to. Thus noop.


Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.utils.build_tensor_info or tf.compat.v1.saved_model.build_tensor_info.
INFO:tensorflow:Assets added to graph.
INFO:tensorflow:Assets written to: /tmp/tfx-interactive-2020-06-18T06_48_24.897236-a9e6u4mw/Transform/transform_graph/4/.temp_path/tftransform_tmp/262d4d55c12240b6a959cbc9c64d05b4/assets
INFO:tensorflow:SavedModel written to: /tmp/tfx-interactive-2020-06-18T06_48_24.897236-a9e6u4mw/Transform/transform_graph/4/.temp_path/tftransform_tmp/262d4d55c12240b6a959cbc9c64d05b4/saved_model.pb


  name: "review"
  type: BYTES
  presence {
    min_fraction: 1.0
    min_count: 1
  }
  shape {
    dim {
      size: 1
    }
  }
}
feature {
  name: "sentiment"
  type: INT
  bool_domain {
  }
  presence {
    min_fraction: 1.0
    min_count: 1
  }
  shape {
    dim {
      size: 1
    }
  }
}
} belongs to. Thus noop.


  name: "review"
  type: BYTES
  presence {
    min_fraction: 1.0
  }
  shape {
    dim {
      size: 1
    }
  }
}
feature {
  name: "sentiment"
  type: INT
  presence {
    min_fraction: 1.0
  }
  shape {
    dim {
      size: 1
    }
  }
}
}), (<PCollection[Analyze/CreateSavedModel/BindTensors/CreateSavedModel/Map(decode).None] at 0x7f30378b6588>, BeamDatasetMetadata(dataset_metadata={'_schema': feature {
  name: "input_mask"
  type: INT
  presence {
    min_fraction: 1.0
  }
  shape {
    dim {
      size: 64
    }
  }
}
feature {
  name: "input_type_ids"
  type: INT
  presence {
    min_fraction: 1.0
  }
  shape {
    dim {
      size: 64
    }
  }
}
feature {
  name: "input_word_ids"
  type: INT
  presence {
    min_fraction: 1.0
  }
  shape {
    dim {
      size: 64
    }
  }
}
feature {
  name: "sentiment"
  type: INT
  presence {
    min_fraction: 1.0
  }
  shape {
    dim {
      size: 1
    }
  }
}
}, deferred_metadata=<PCollection[Analyze/ComputeDeferredMetadata.None] at 0

  name: "review"
  type: BYTES
  presence {
    min_fraction: 1.0
  }
  shape {
    dim {
      size: 1
    }
  }
}
feature {
  name: "sentiment"
  type: INT
  presence {
    min_fraction: 1.0
  }
  shape {
    dim {
      size: 1
    }
  }
}
}), (<PCollection[Analyze/CreateSavedModel/BindTensors/CreateSavedModel/Map(decode).None] at 0x7f30378b6588>, BeamDatasetMetadata(dataset_metadata={'_schema': feature {
  name: "input_mask"
  type: INT
  presence {
    min_fraction: 1.0
  }
  shape {
    dim {
      size: 64
    }
  }
}
feature {
  name: "input_type_ids"
  type: INT
  presence {
    min_fraction: 1.0
  }
  shape {
    dim {
      size: 64
    }
  }
}
feature {
  name: "input_word_ids"
  type: INT
  presence {
    min_fraction: 1.0
  }
  shape {
    dim {
      size: 64
    }
  }
}
feature {
  name: "sentiment"
  type: INT
  presence {
    min_fraction: 1.0
  }
  shape {
    dim {
      size: 1
    }
  }
}
}, deferred_metadata=<PCollection[Analyze/ComputeDeferredMetadata.None] at 0

value: "\n\016\n\014asset_path:0\022\tvocab.txt"

value: "\n\016\n\014asset_path:0\022\tvocab.txt"

value: "\n\016\n\014asset_path:0\022\tvocab.txt"



ExecutionResult(
    component_id: Transform
    execution_id: 4
    outputs:
        transform_graph: Channel(
            type_name: TransformGraph
            artifacts: [Artifact(type_name: TransformGraph, uri: /tmp/tfx-interactive-2020-06-18T06_48_24.897236-a9e6u4mw/Transform/transform_graph/4, id: 5)]
        )
        transformed_examples: Channel(
            type_name: Examples
            artifacts: [Artifact(type_name: Examples, uri: /tmp/tfx-interactive-2020-06-18T06_48_24.897236-a9e6u4mw/Transform/transformed_examples/4, id: 6)]
        ))

In [15]:
from tfx_bsl.coders.example_coder import ExampleToNumpyDict

pp = pprint.PrettyPrinter()

# Get the URI of the output artifact representing the transformed examples, which is a directory
train_uri = transform.outputs['transformed_examples'].get()[0].uri

print(train_uri)

# Get the list of files in this directory (all compressed TFRecord files)
tfrecord_folders = [os.path.join(train_uri, name) for name in os.listdir(train_uri)]
tfrecord_filenames = []
for tfrecord_folder in tfrecord_folders:
    for name in os.listdir(tfrecord_folder):
        tfrecord_filenames.append(os.path.join(tfrecord_folder, name))


# Create a TFRecordDataset to read these files
dataset = tf.data.TFRecordDataset(tfrecord_filenames, compression_type="GZIP")

for tfrecord in dataset.take(1):
    serialized_example = tfrecord.numpy()
    example = ExampleToNumpyDict(serialized_example)
    pp.pprint(example)

/tmp/tfx-interactive-2020-06-18T06_48_24.897236-a9e6u4mw/Transform/transformed_examples/4
{'input_mask': array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]),
 'input_type_ids': array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 'input_word_ids': array([  101,  2056,  2428, 16713,  2034,  2465, 10874, 18481,  5020,
        8310, 23873,  2440, 10676, 27890, 17796,  2135, 27347,  2143,
        2296,  6987,  3849,  2443,  3114,  2130,  2295,  5436, 15839,
        3084,  3168,  2172,  2196,  2130,  8280,  7863,  5450,  3568,
        2187,  6233, 14480, 29461, 13687, 21983, 14955,  6962,  3211,
       28997,  3907,  2158,  4663,  4545,  2349,  3025, 16713,  5920,
       

In [17]:
%%skip_for_export
%%writefile trainer.py

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_model_analysis as tfma
import tensorflow_transform as tft
from tensorflow_transform.tf_metadata import schema_utils

from typing import Text

import absl
import tensorflow as tf
from tensorflow import keras
import tensorflow_transform as tft
from tfx.components.trainer.executor import TrainerFnArgs


_LABEL_KEY = 'sentiment'
BERT_TFHUB_URL = "https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/2"


def _gzip_reader_fn(filenames):
    """Small utility returning a record reader that can read gzip'ed files."""
    return tf.data.TFRecordDataset(filenames, compression_type='GZIP')

def load_bert_layer(model_url=BERT_TFHUB_URL):
    # Load the pre-trained BERT model as layer in Keras
    bert_layer = hub.KerasLayer(
        handle=model_url,
        trainable=False)  # model can be fine-tuned 
    return bert_layer

def get_model(tf_transform_output, max_seq_length=64, num_labels=2):

    # dynamically create inputs for all outputs of our transform graph
    feature_spec = tf_transform_output.transformed_feature_spec()  
    feature_spec.pop(_LABEL_KEY)

    inputs = {
        key: tf.keras.layers.Input(shape=(max_seq_length), name=key, dtype=tf.int64)
            for key in feature_spec.keys()
    }

    input_word_ids = tf.cast(inputs["input_word_ids"], dtype=tf.int32)
    input_mask = tf.cast(inputs["input_mask"], dtype=tf.int32)
    input_type_ids = tf.cast(inputs["input_type_ids"], dtype=tf.int32)

    bert_layer = load_bert_layer()
    pooled_output, _ = bert_layer(
        [input_word_ids, 
         input_mask, 
         input_type_ids
        ]
    )
    
    # Add additional layers depending on your problem
    x = tf.keras.layers.Dense(256, activation='relu')(pooled_output)
    dense = tf.keras.layers.Dense(64, activation='relu')(x)
    pred = tf.keras.layers.Dense(1, activation='sigmoid')(dense)

    keras_model = tf.keras.Model(
        inputs=[
                inputs['input_word_ids'], 
                inputs['input_mask'], 
                inputs['input_type_ids']], 
        outputs=pred)
    keras_model.compile(loss='binary_crossentropy', 
                        optimizer=tf.keras.optimizers.Adam(learning_rate=0.01), 
                        metrics=['accuracy']
                        )
    return keras_model


def _get_serve_tf_examples_fn(model, tf_transform_output):
    """Returns a function that parses a serialized tf.Example and applies TFT."""

    model.tft_layer = tf_transform_output.transform_features_layer()

    @tf.function
    def serve_tf_examples_fn(serialized_tf_examples):
        """Returns the output to be used in the serving signature."""
        feature_spec = tf_transform_output.raw_feature_spec()
        feature_spec.pop(_LABEL_KEY)
        parsed_features = tf.io.parse_example(serialized_tf_examples, feature_spec)

        transformed_features = model.tft_layer(parsed_features)

        outputs = model(transformed_features)
        return {'outputs': outputs}

    return serve_tf_examples_fn

def _input_fn(file_pattern: Text,
              tf_transform_output: tft.TFTransformOutput,
              batch_size: int = 32) -> tf.data.Dataset:
    """Generates features and label for tuning/training.

    Args:
      file_pattern: input tfrecord file pattern.
      tf_transform_output: A TFTransformOutput.
      batch_size: representing the number of consecutive elements of returned
        dataset to combine in a single batch

    Returns:
      A dataset that contains (features, indices) tuple where features is a
        dictionary of Tensors, and indices is a single Tensor of label indices.
    """
    transformed_feature_spec = (
        tf_transform_output.transformed_feature_spec().copy())

    dataset = tf.data.experimental.make_batched_features_dataset(
        file_pattern=file_pattern,
        batch_size=batch_size,
        features=transformed_feature_spec,
        reader=_gzip_reader_fn,
        label_key=_LABEL_KEY)

    return dataset

# TFX Trainer will call this function.
def run_fn(fn_args: TrainerFnArgs):
    """Train the model based on given args.

    Args:
      fn_args: Holds args used to train the model as name/value pairs.
    """
    tf_transform_output = tft.TFTransformOutput(fn_args.transform_output)
    print (fn_args.train_files)
    print (fn_args.eval_files)
    train_dataset = _input_fn(fn_args.train_files, tf_transform_output, 32)
    eval_dataset = _input_fn(fn_args.eval_files, tf_transform_output, 32)

    mirrored_strategy = tf.distribute.MirroredStrategy()
    with mirrored_strategy.scope():
        model = get_model(tf_transform_output=tf_transform_output)

    model.fit(
        train_dataset,
        steps_per_epoch=fn_args.train_steps,
        validation_data=eval_dataset,
        validation_steps=fn_args.eval_steps)

    signatures = {
        'serving_default':
            _get_serve_tf_examples_fn(model,
                                      tf_transform_output).get_concrete_function(
                                          tf.TensorSpec(
                                              shape=[None],
                                              dtype=tf.string,
                                              name='examples')),
    }
    model.save(fn_args.serving_model_dir, save_format='tf', signatures=signatures)


Overwriting trainer.py
This cell will be skipped during export to pipeline.


In [18]:
# NOTE: Adjust the number of training and evaluation steps
TRAINING_STEPS = 1000
EVALUATION_STEPS = 100

trainer = Trainer(
    module_file=os.path.abspath("trainer.py"),
    custom_executor_spec=executor_spec.ExecutorClassSpec(GenericExecutor),
    examples=transform.outputs['transformed_examples'],
    transform_graph=transform.outputs['transform_graph'],
    schema=schema_gen.outputs['schema'],
    train_args=trainer_pb2.TrainArgs(num_steps=TRAINING_STEPS),
    eval_args=trainer_pb2.EvalArgs(num_steps=EVALUATION_STEPS))
context.run(trainer)

['/tmp/tfx-interactive-2020-06-18T06_48_24.897236-a9e6u4mw/Transform/transformed_examples/4/train/*']
['/tmp/tfx-interactive-2020-06-18T06_48_24.897236-a9e6u4mw/Transform/transformed_examples/4/eval/*']
INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)
1000/1000 [==============================] - 7360s 7s/step - loss: 0.7088 - accuracy: 0.4998 - val_loss: 0.6941 - val_accuracy: 0.4775
INFO:tensorflow:Assets written to: /tmp/tfx-interactive-2020-06-18T06_48_24.897236-a9e6u4mw/Trainer/model/5/serving_model_dir/assets


ExecutionResult(
    component_id: Trainer
    execution_id: 5
    outputs:
        model: Channel(
            type_name: Model
            artifacts: [Artifact(type_name: Model, uri: /tmp/tfx-interactive-2020-06-18T06_48_24.897236-a9e6u4mw/Trainer/model/5, id: 7)]
        ))

In [19]:
model_resolver = ResolverNode(
    instance_name='latest_blessed_model_resolver',
    resolver_class=latest_blessed_model_resolver.LatestBlessedModelResolver,
    model=Channel(type=Model),
    model_blessing=Channel(type=ModelBlessing))

context.run(model_resolver)

ExecutionResult(
    component_id: ResolverNode.latest_blessed_model_resolver
    execution_id: 6
    outputs:
        model: Channel(
            type_name: Model
            artifacts: []
        )
        model_blessing: Channel(
            type_name: ModelBlessing
            artifacts: []
        ))

In [20]:
eval_config = tfma.EvalConfig(
    model_specs=[
        tfma.ModelSpec(label_key='label')
    ],
    metrics_specs=[
        tfma.MetricsSpec(
            metrics=[
                tfma.MetricConfig(class_name='ExampleCount')
            ],
            thresholds = {
                'binary_accuracy': tfma.MetricThreshold(
                    value_threshold=tfma.GenericValueThreshold(
                        lower_bound={'value': 0.5}),
                    change_threshold=tfma.GenericChangeThreshold(
                       direction=tfma.MetricDirection.HIGHER_IS_BETTER,
                       absolute={'value': -1e-10}))
            }
        )
    ],
    slicing_specs=[
        # An empty slice spec means the overall slice, i.e. the whole dataset.
        tfma.SlicingSpec(),
    ])

evaluator = Evaluator(
    examples=example_gen.outputs['examples'],
    model=trainer.outputs['model'],
    baseline_model=model_resolver.outputs['model'],
    eval_config=eval_config
)

context.run(evaluator)

/usr/local/lib/python3.6/dist-packages/apache_beam/transforms/util.py:383: RuntimeWarning: divide by zero encountered in true_divide
  cook_ds = 0.5 / s2 * errs**2 * (h / (1 - h)**2)
/usr/local/lib/python3.6/dist-packages/apache_beam/transforms/util.py:422: RankWarning: Polyfit may be poorly conditioned
  a, b = self.linear_regression(xs, ys)
/usr/local/lib/python3.6/dist-packages/apache_beam/transforms/util.py:422: RankWarning: Polyfit may be poorly conditioned
  a, b = self.linear_regression(xs, ys)
/usr/local/lib/python3.6/dist-packages/apache_beam/transforms/util.py:422: RankWarning: Polyfit may be poorly conditioned
  a, b = self.linear_regression(xs, ys)
/usr/local/lib/python3.6/dist-packages/apache_beam/transforms/util.py:422: RankWarning: Polyfit may be poorly conditioned
  a, b = self.linear_regression(xs, ys)
/usr/local/lib/python3.6/dist-packages/apache_beam/transforms/util.py:422: RankWarning: Polyfit may be poorly conditioned
  a, b = self.linear_regression(xs, ys)
/usr/lo

ExecutionResult(
    component_id: Evaluator
    execution_id: 7
    outputs:
        evaluation: Channel(
            type_name: ModelEvaluation
            artifacts: [Artifact(type_name: ModelEvaluation, uri: /tmp/tfx-interactive-2020-06-18T06_48_24.897236-a9e6u4mw/Evaluator/evaluation/7, id: 8)]
        )
        blessing: Channel(
            type_name: ModelBlessing
            artifacts: [Artifact(type_name: ModelBlessing, uri: /tmp/tfx-interactive-2020-06-18T06_48_24.897236-a9e6u4mw/Evaluator/blessing/7, id: 9)]
        ))

In [21]:
# Check the blessing
!ls {evaluator.outputs['blessing'].get()[0].uri}

BLESSED


In [22]:
!mkdir /content/serving_model_dir

serving_model_dir = "/content/serving_model_dir"

pusher = Pusher(
    model=trainer.outputs['model'],
    model_blessing=evaluator.outputs['blessing'],
    push_destination=pusher_pb2.PushDestination(
        filesystem=pusher_pb2.PushDestination.Filesystem(
            base_directory=serving_model_dir)))

context.run(pusher)

mkdir: cannot create directory ‘/content/serving_model_dir’: File exists


ExecutionResult(
    component_id: Pusher
    execution_id: 8
    outputs:
        pushed_model: Channel(
            type_name: PushedModel
            artifacts: [Artifact(type_name: PushedModel, uri: /tmp/tfx-interactive-2020-06-18T06_48_24.897236-a9e6u4mw/Pusher/pushed_model/8, id: 10)]
        ))

In [24]:
pusher.outputs.model_push.get()[0].uri

'/tmp/tfx-interactive-2020-06-18T06_48_24.897236-a9e6u4mw/Pusher/pushed_model/8'

In [39]:
def _bytes_feature(value):
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

push_uri = pusher.outputs.model_push.get()[0].uri
latest_version = os.listdir(push_uri)[1]
latest_version_path = os.path.join(push_uri, latest_version)
loaded_model = tf.saved_model.load(push_uri)

example_str = b"This is the finest show ever produced for TV. Each episode is a triumph. The casting, the writing, the timing are all second to none. This cast performs miracles."
example = tf.train.Example(features=tf.train.Features(feature={
    'review': _bytes_feature(example_str)}))

serialized_example = example.SerializeToString()
f = loaded_model.signatures["serving_default"]
print(f(tf.constant([serialized_example])))

{'outputs': <tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[0.50781095]], dtype=float32)>}
